In [11]:
import numpy as np
import xarray as xr

# Chlorophyll

In [ ]:
import glob
import dask
import re
import pandas as pd
from tqdm import tqdm

# Define the path where files are present
chl_path = "data/CHL/v6_files/"

# Creates a list of the filenames in the folder
files = sorted(glob.glob(chl_path + "ESACCI-OC-L3S-CHLOR_A-MERGED-1D_DAILY_4km_GEO_PML_OCx-*-fv6.0.nc"))

In [4]:
# Initialize an empty list to store Dask arrays
arrays = []

# Loop through each file, load, and append to the list
for file in tqdm(files):
    ds = xr.open_dataset(file, 
                         chunks={'lat': -1, 'lon': -1})
    arrays.append(dask.array.asarray(ds.chlor_a.data, chunks=ds.chlor_a.data.chunksize)\
                .reshape((1,4320,8640)))
    #arrays.append(ds)

100%|██████████| 7670/7670 [09:51<00:00, 12.97it/s] 


In [ ]:
# Use Dask to concatenate along the time dimension
chl = dask.array.concatenate(arrays, axis = 0)

np.save('data/')

In [9]:
# We will use the same longitude and latitude references from Chl product
# Difference is about 1e-5 degrees
lon = np.load("./data/processed/ref_lon.npy")
lat = np.load("./data/processed/ref_lat.npy")[::-1] # latitudes are inverted

# Creates the time coordinate from the file names
timestamps = [re.search(r'\d{8}', file).group() for file in files]  # Assuming a YYYYMMDD format
time_coord = pd.to_datetime(timestamps, format='%Y%m%d')

# Use Dask to concatenate along the time dimension
chl = xr.DataArray(chl, 
                   coords = [time_coord, lat, lon], 
                   dims = ["time", "latitude", "longitude"])\
    .sortby("latitude").convert_calendar("noleap")

chl["time"] = chl.indexes["time"].to_datetimeindex()
chl.name = 'chl'


/tmp/ipykernel_1230802/3592293640.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  chl["time"] = chl.indexes["time"].to_datetimeindex()


In [10]:
chl.to_zarr('./data/processed/chl_2003-2023.zarr')

In [19]:
chl = xr.open_zarr('./data/processed/chl_2003-2023.zarr')
num_years = len(chl.time) // 365
chl = chl.assign_coords({'dayofyear' : ('time', np.tile(np.arange(0, 365), num_years))})

clima = chl.groupby('dayofyear').mean()
clima.to_zarr('./data/processed/climatologies/chl_clima.zarr')

# SST

The main difference between OC-CCI product for Chl-a and MODIS SST is that MODIS data are provided in a single-file fashion. This means that each file contains the state of SST for a single day. For our purposes, we need to merge all these files in a single one (like Chl-a). However, this work is not well-handled by xarray alone.

In [38]:
import glob
import dask
import re
import pandas as pd
from tqdm import tqdm

# Define the path where files are present
sst_path = "./data/SST/original_files/"

# Creates a list of the filenames in the folder
files = sorted(glob.glob(sst_path + "AQUA_MODIS.*.L3m.DAY.SST4.sst4.4km.nc"))

In [39]:
# Initialize an empty list to store Dask arrays
arrays = []

# Loop through each file, load, and append to the list
for file in tqdm(files):
    ds = xr.open_dataset(file, 
                         chunks={'lat': -1, 'lon': -1})
    arrays.append(dask.array.asarray(ds.sst4.data, chunks=ds.sst4.data.chunksize)\
                .reshape((1,4320,8640)))
    #arrays.append(ds)

100%|██████████| 7655/7655 [01:12<00:00, 106.10it/s]


In [73]:
# Use Dask to concatenate along the time dimension
sst = dask.array.concatenate(arrays, axis = 0)

In [74]:
# We will use the same longitude and latitude references from Chl product
# Difference is about 1e-5 degrees
lon = np.load("./data/processed/ref_lon.npy")
lat = np.load("./data/processed/ref_lat.npy")[::-1] # latitudes are inverted

# Creates the time coordinate from the file names
timestamps = [re.search(r'\d{8}', file).group() for file in files]  # Assuming a YYYYMMDD format
time_coord = pd.to_datetime(timestamps, format='%Y%m%d')

# Use Dask to concatenate along the time dimension
sst = xr.DataArray(sst, 
                   coords = [time_coord, lat, lon], 
                   dims = ["time", "latitude", "longitude"])\
                .sortby("latitude")\
                .reindex(time = pd.date_range(time_coord.min(), time_coord.max(), freq = 'D'))\
                .convert_calendar('noleap')\
                .chunk({'time' : 1})

sst["time"] = sst.indexes["time"].to_datetimeindex()
sst.name = 'sst'


/data01/benassi/miniconda3/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/data01/benassi/miniconda3/lib/python3.11/site-packages/xarray/core/indexing.py:1446: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/tmp/ipykernel

In [75]:
sst

<xarray.DataArray 'sst' (time: 7665, latitude: 4320, longitude: 8640)>
dask.array<rechunk-merge, shape=(7665, 4320, 8640), dtype=float32, chunksize=(1, 4320, 8640), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -89.98 -89.94 -89.9 ... 89.9 89.94 89.98
  * longitude  (longitude) float64 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time       (time) datetime64[ns] 2003-01-01 2003-01-02 ... 2023-12-31

In [77]:
sst.to_zarr('./data/processed/sst_2003-2023.zarr')

In [11]:
sst.to_netcdf("./data/processed/sst.nc")

In [38]:
del sst

# KD490

Same computation of SST for MODIS-Aqua KD490 data. The original work was using GlobColour merged data, but the CMEMS service is migrating download procedures and I couldn't download the dataset without errors or freezing Calypso. I will use KD490 by MODIS until the end of March 2024 (supposed end of CMEMS migration work), then we'll see.
I also included the OC-CCI product (https://www.mdpi.com/1424-8220/19/19/4285), which improves cloud cover and it's the same product of chlorophyll. OC-CCI data were downloaded from https://www.oceancolour.org/

In [39]:
import glob
import dask
import re
import pandas as pd
from tqdm import tqdm

# Define the path where files are present
#kd_path = "./data/KD/original_files/" # Old MODIS files
kd_path = "./data/KD/OC-CCI_files/"

# Creates a list of the filenames in the folder
files = sorted(glob.glob(kd_path + "ESACCI-OC-L3S-K_490-MERGED-1D_DAILY_4km_GEO_PML_KD490_Lee-*-fv6.0.nc"))

In [41]:
# Initialize an empty list to store Dask arrays
arrays = []

# Loop through each file, load, and append to the list
for file in tqdm(files):
    ds = xr.open_dataset(file, chunks={'lat': -1, 'lon': -1})
    arrays.append(dask.array.asarray(ds.kd_490.data, chunks=ds.kd_490.data.chunksize)\
                 .reshape((1,4320,8640)))

  0%|          | 0/7670 [00:00<?, ?it/s]

100%|██████████| 7670/7670 [20:36<00:00,  6.20it/s]


In [42]:
# Use Dask to concatenate along the time dimension
kd = dask.array.concatenate(arrays, axis = 0)

kd

dask.array<concatenate, shape=(7670, 4320, 8640), dtype=float32, chunksize=(1, 4320, 8640), chunktype=numpy.ndarray>

In [43]:
# We will use the same longitude and latitude references from Chl product
# Difference is about 1e-5 degrees
lon = np.load("./data/processed/ref_lon.npy")
lat = np.load("./data/processed/ref_lat.npy")[::-1] # latitudes are inverted

# Creates the time coordinate from the file names
timestamps = [re.search(r'\d{8}', file).group() for file in files]  # Assuming a YYYYMMDD format
time_coord = pd.to_datetime(timestamps, format='%Y%m%d')

# Use Dask to concatenate along the time dimension
kd = xr.DataArray(kd, 
                   coords = [time_coord, lat, lon], 
                   dims = ["time", "latitude", "longitude"])\
    .sortby("latitude").convert_calendar("noleap")

kd["time"] = kd.indexes["time"].to_datetimeindex()


/tmp/ipykernel_736241/2240902811.py:16: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  kd["time"] = kd.indexes["time"].to_datetimeindex()


We actually need the estimation of the euphotic depth for our work, thus the estimated value dataset is directly created (saving future computations and memory)

In [45]:
# Estimation of euphotic depth from Kirk (1994) (see my thesis)
eu_depth = 4.6 / kd
eu_depth.name = 'euphotic_depth'

del kd
#eu_depth.to_netcdf("./data/processed/eudepth_occci.nc")

In [49]:
eu_depth.to_zarr("./data/processed/eudepth_2003-2023.zarr")

In [18]:
del kd
del eu_depth

# Bathymetry interpolation 

In [75]:
bathy = xr.open_dataset("./data/bathy/bathymetry.nc")\
        .elevation \
        .rename({"lon" : "longitude", "lat" : "latitude"})

In [79]:
lon = np.load("./data/processed/ref_lon.npy")
lat = np.load("./data/processed/ref_lat.npy")

bathy = bathy.interp(longitude = lon,
            latitude = lat,
            method = "nearest")

In [81]:
bathy.to_netcdf("./data/processed/bathymetry.nc")